In [1]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader

Input the API token you got from huggingface.

In [2]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

In [3]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

For the LLM we choose Qwen2.5-72B.

In [4]:
repo_id = "Qwen/Qwen2.5-72B-Instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    max_new_tokens=2000
)

llm = ChatHuggingFace(llm=llm)

Then we set a simple answer template and do a small test of our choosen LLM.

In [5]:
# Create an answer question template.
template = """Answer the question below:
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()
llm_chain = prompt | llm | output_parser

# test
question = "Who is KIM Chungha?"
response = llm_chain.invoke({"question": question})
print(response)

KIM Chungha, often known simply as Chungha, is a South Korean singer, dancer, and actress. Born on October 5, 1996, in Gwangmyeong, South Korea, Chungha gained initial fame as a contestant on the survival show " Produce 101" in 2016, where she finished in fourth place and became a member of the temporary girl group I.O.I. Following the group's disbandment, Chungha pursued a solo career and has since released several successful singles and mini-albums, including "Gotta Go," "Why Don't You Love Me," and "Snapping."

Chungha is known for her powerful vocal abilities, dynamic dance performances, and strong stage presence. She has also acted in a few television dramas, such as "The Player" (2017) and "Shining Queen" (2020). Her music often blends elements of pop, R&B, and EDM, and she has a dedicated fanbase both in South Korea and internationally.


After that we set up our knowledge base which based on a pdf of a Korean singer Kim Chungha.
For the embedding model we choose sentence-transformers/all-MiniLM-l6-v2, and we use FAISS as our vector database.

In [6]:
# Load PDF
loader = PyPDFLoader("Chungha.pdf")
docs = loader.load_and_split()

# Embedding model
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINGFACEHUB_API_TOKEN,
    model_name="sentence-transformers/all-MiniLM-l6-v2"
)

# Segment the document and create Vector Embeddings
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector_store = FAISS.from_documents(documents, embeddings)

In [7]:
# Create prompt template
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context:

Question: {input}

Context: {context}
""")

# Create document chain and retriever chain
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Define generate response function
def generate_response(query):
    response = retrieval_chain.invoke({"input": query})
    return response["answer"].strip()

Test our RAG system.

In [8]:
# Test the LLM under RAG
query = "When did Roller Coaster come out?"
print(generate_response(query))

The song "Roller Coaster" was released on January 17, 2018, as the title track of Chung Ha's second extended play (EP) titled "Offset."
